<a href="https://colab.research.google.com/github/toruuno/DNN_d1d2/blob/master/1_2_back_propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

## Googleドライブのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## sys.pathの設定

以下では，Googleドライブのマイドライブ直下にDNN_codeフォルダを置くことを仮定しています．必要に応じて，パスを変更してください．

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/DNN_code')
sys.path.append('/content/drive/My Drive/common')
sys.path.append('/content/drive/My Drive')

# importと関数定義

In [ ]:
import numpy as np
from common import functions
import matplotlib.pyplot as plt

def print_vec(text, vec):
    print("*** " + text + " ***")
    print(vec)
    #print("shape: " + str(x.shape))
    print("")


# メインプログラム

In [ ]:
# ウェイトとバイアスを設定
# ネートワークを作成
def init_network(debug=True):
    if debug:
      print("##### ネットワークの初期化 #####")

    network = {}
    network['W1'] = np.array([
        [0.1, 0.3, 0.5],
        [0.2, 0.4, 0.6]
    ])

    network['W2'] = np.array([
        [0.1, 0.4],
        [0.2, 0.5],
        [0.3, 0.6]
    ])

    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['b2'] = np.array([0.1, 0.2])
    
    if debug:
      print_vec("重み1", network['W1'])
      print_vec("重み2", network['W2'])
      print_vec("バイアス1", network['b1'])
      print_vec("バイアス2", network['b2'])

    return network

# 順伝播
def forward(network, x, debug=True):
    if debug:
      print("##### 順伝播開始 #####")

    W1, W2 = network['W1'], network['W2']
    b1, b2 = network['b1'], network['b2']
    
    u1 = np.dot(x, W1) + b1
    z1 = functions.relu(u1)
    u2 = np.dot(z1, W2) + b2
    y = functions.softmax(u2)
    
    if debug:
      print_vec("総入力1", u1)
      print_vec("中間層出力1", z1)
      print_vec("総入力2", u2)
      print_vec("出力1", y)
      print("出力合計: " + str(np.sum(y)))

    return y, z1

# 誤差逆伝播
def backward(x, d, z1, y, debug=True):
    if debug:
      print("\n##### 誤差逆伝播開始 #####")

    grad = {}

    W1, W2 = network['W1'], network['W2']
    b1, b2 = network['b1'], network['b2']
    #  出力層でのデルタ
    delta2 = functions.d_sigmoid_with_loss(d, y)
    #  b2の勾配
    grad['b2'] = np.sum(delta2, axis=0)
    #  W2の勾配
    grad['W2'] = np.dot(z1.T, delta2)
    #  中間層でのデルタ
    delta1 = np.dot(delta2, W2.T) * functions.d_relu(z1)
    # b1の勾配
    grad['b1'] = np.sum(delta1, axis=0)
    #  W1の勾配
    grad['W1'] = np.dot(x.T, delta1)
        
    if debug:
      print_vec("偏微分_dE/du2", delta2)
      print_vec("偏微分_dE/du2", delta1)

      print_vec("偏微分_重み1", grad["W1"])
      print_vec("偏微分_重み2", grad["W2"])
      print_vec("偏微分_バイアス1", grad["b1"])
      print_vec("偏微分_バイアス2", grad["b2"])

    return grad
    
# 訓練データ
x = np.array([[1.0, 5.0]])
# 目標出力
d = np.array([[0, 1]])


#課題設定：
#学習率を弄り誤差の減り具合を評価
#結果的には学習率が高いほど誤差の収束は進むという程度
#(過学習か如何かはここでは一切気にしない)

#  学習率
learning_rates = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001]
#learning_rate = 0.01
debugNow=False

for learning_rate in learning_rates:
  print("----------------------------")
  print("学習率:{}".format(learning_rate))
  network =  init_network(debug=debugNow)
  y, z1 = forward(network, x, debug=debugNow)

  # 誤差
  loss_before = functions.cross_entropy_error(d, y)

  grad = backward(x, d, z1, y, debug=debugNow)
  for key in ('W1', 'W2', 'b1', 'b2'):
      network[key]  -= learning_rate * grad[key]

  print("##### 結果表示 #####")    
  y, z1 = forward(network, x, debug=debugNow)

  # 誤差
  loss_after = functions.cross_entropy_error(d, y)
  print("誤差:before={}  after={}".format(loss_before, loss_after))

  if debugNow:
    print("##### 更新後パラメータ #####") 
    print_vec("重み1", network['W1'])
    print_vec("重み2", network['W2'])
    print_vec("バイアス1", network['b1'])
    print_vec("バイアス2", network['b2'])


----------------------------
学習率:100
##### 結果表示 #####
誤差:before=0.09109133135793131  after=-9.999999505838704e-08
----------------------------
学習率:10
##### 結果表示 #####
誤差:before=0.09109133135793131  after=-9.999999505838704e-08
----------------------------
学習率:1
##### 結果表示 #####
誤差:before=0.09109133135793131  after=0.00035840211750155017
----------------------------
学習率:0.1
##### 結果表示 #####
誤差:before=0.09109133135793131  after=0.05759133192915599
----------------------------
学習率:0.01
##### 結果表示 #####
誤差:before=0.09109133135793131  after=0.08710960847142761
----------------------------
学習率:0.001
##### 結果表示 #####
誤差:before=0.09109133135793131  after=0.0906861813303538
----------------------------
学習率:0.0001
##### 結果表示 #####
誤差:before=0.09109133135793131  after=0.09105074589590591
